In [14]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
from langchain.retrievers import WikipediaRetriever
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_community.tools import DuckDuckGoSearchResults
import yfinance
import json

def get_ticker(inputs):
    company_name = inputs["company_name"] #inputs에는 {"company_name": "Apple"} 이런식으로 들어옴
    # ddg = DuckDuckGoSearchResults()
    ddg = DuckDuckGoSearchAPIWrapper()
    # return ddg.text(f"Ticker symbol of {company_name}")
    return {"ticker": "AAPL"}

    # wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    # return wikipedia.run(f"Ticker symbol of {company_name} example: AAPL")

def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    # return stock.income_stmt #pandas dataframe
    return json.dumps(stock.income_stmt.to_json()) #json string

def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())

def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json()) #json string


functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

# a = get_income_statement({"ticker": "AAPL"})
# print(type(a))
# print(a)


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from openai import OpenAI

client = OpenAI()

assistant = client.beta.assistants.create(
    name="Investor Assistant",
    instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
    model="gpt-4-1106-preview",
    tools=functions,
)

print(assistant)
# id='asst_CtI9FmsHF9VLke9iz8pZ3KVw''

Assistant(id='asst_CtI9FmsHF9VLke9iz8pZ3KVw', created_at=1737851182, description=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', metadata={}, model='gpt-4-1106-preview', name='Investor Assistant', object='assistant', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given a ticker symbol (i.e AAPL) returns the company's income statement.", parameters={'type': 'object', 'properties': {'ticker': {'type': 'string', 'description': 'Ticker symbol of the company'}}, 'required': ['ticker']}, strict=False), type='function'), FunctionTool(function=Funct

In [ ]:
# print(get_ticker({"company_name":"Apple"}))

In [17]:
# ticker = {"ticker": "NET"}
ticker = get_ticker({"company_name":"Apple"})

get_income_statement(ticker)

'"{\\"1727654400000\\":{\\"Tax Effect Of Unusual Items\\":0.0,\\"Tax Rate For Calcs\\":0.240912,\\"Normalized EBITDA\\":134661000000.0,\\"Net Income From Continuing Operation Net Minority Interest\\":93736000000.0,\\"Reconciled Depreciation\\":11445000000.0,\\"Reconciled Cost Of Revenue\\":210352000000.0,\\"EBITDA\\":134661000000.0,\\"EBIT\\":123216000000.0,\\"Net Interest Income\\":null,\\"Interest Expense\\":null,\\"Interest Income\\":null,\\"Normalized Income\\":93736000000.0,\\"Net Income From Continuing And Discontinued Operation\\":93736000000.0,\\"Total Expenses\\":267819000000.0,\\"Total Operating Income As Reported\\":123216000000.0,\\"Diluted Average Shares\\":null,\\"Basic Average Shares\\":null,\\"Diluted EPS\\":null,\\"Basic EPS\\":null,\\"Diluted NI Availto Com Stockholders\\":93736000000.0,\\"Net Income Common Stockholders\\":93736000000.0,\\"Net Income\\":93736000000.0,\\"Net Income Including Noncontrolling Interests\\":93736000000.0,\\"Net Income Continuous Operations\

In [15]:
get_daily_stock_performance(ticker)

'"{\\"Open\\":{\\"1729828800000\\":89.8199996948,\\"1730088000000\\":89.3799972534,\\"1730174400000\\":88.4400024414,\\"1730260800000\\":89.5,\\"1730347200000\\":88.2900009155,\\"1730433600000\\":88.6100006104,\\"1730696400000\\":87.0,\\"1730782800000\\":86.9199981689,\\"1730869200000\\":90.25,\\"1730955600000\\":94.8899993896,\\"1731042000000\\":88.4499969482,\\"1731301200000\\":91.25,\\"1731387600000\\":92.8899993896,\\"1731474000000\\":91.4199981689,\\"1731560400000\\":94.4599990845,\\"1731646800000\\":90.4300003052,\\"1731906000000\\":89.9749984741,\\"1731992400000\\":91.4649963379,\\"1732078800000\\":96.5599975586,\\"1732165200000\\":99.7600021362,\\"1732251600000\\":104.0,\\"1732510800000\\":106.75,\\"1732597200000\\":104.0,\\"1732683600000\\":101.8799972534,\\"1732856400000\\":99.5599975586,\\"1733115600000\\":106.0299987793,\\"1733202000000\\":103.3700027466,\\"1733288400000\\":107.0,\\"1733374800000\\":109.4100036621,\\"1733461200000\\":111.2799987793,\\"1733720400000\\":112.9

In [16]:
get_balance_sheet(ticker)

'"{\\"1703980800000\\":{\\"Treasury Shares Number\\":0.0,\\"Ordinary Shares Number\\":337532000.0,\\"Share Issued\\":337532000.0,\\"Net Debt\\":1196498000.0,\\"Total Debt\\":1435203000.0,\\"Tangible Book Value\\":595436000.0,\\"Invested Capital\\":2046409000.0,\\"Working Capital\\":1415993000.0,\\"Net Tangible Assets\\":595436000.0,\\"Capital Lease Obligations\\":151841000.0,\\"Common Stock Equity\\":763047000.0,\\"Total Capitalization\\":2046409000.0,\\"Total Equity Gross Minority Interest\\":763047000.0,\\"Stockholders Equity\\":763047000.0,\\"Other Equity Interest\\":null,\\"Gains Losses Not Affecting Retained Earnings\\":1984000.0,\\"Other Equity Adjustments\\":1984000.0,\\"Retained Earnings\\":-1023840000.0,\\"Additional Paid In Capital\\":1784566000.0,\\"Capital Stock\\":337000.0,\\"Common Stock\\":337000.0,\\"Total Liabilities Net Minority Interest\\":1996720000.0,\\"Total Non Current Liabilities Net Minority Interest\\":1429636000.0,\\"Other Non Current Liabilities\\":15540000.